In [1]:
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt

# hyper parameters
learning_rate = 0.001
training_epochs = 50

# input place holders
training_data = np.load('train_data.npy')
testing_data = np.load('test_data.npy')

x_data_test = testing_data[:, 0:-1]
y_data_test = testing_data[:, [-1]]

x_data = training_data[:, 0:-1]
y_data = training_data[:, [-1]]

X = tf.placeholder(tf.float32, [None,12288])
X_img = tf.reshape(X, [-1, 64, 64, 3])   # img 64x64x3 (color)
Y = tf.placeholder(tf.int32, [None, 1])

Y_one_hot = tf.one_hot(Y, 2)  # one hot
Y_one_hot = tf.reshape(Y_one_hot, [-1, 2])

# L1 ImgIn shape=(?, 64, 64, 3)
W1 = tf.Variable(tf.random_normal([3, 3, 3, 128], stddev=0.01))
#    Conv     -> (?, 32, 32, 128)
#    Pool     -> (?, 32, 32, 128)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')


# L2 ImgIn shape=(?, 32, 32, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 128, 64], stddev=0.01))
#    Conv      ->(?, 32, 32, 64)
#    Pool      ->(?, 16, 16, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

# L3 
W3 = tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=0.01))
#    Conv      ->(?, 16, 16, 64)
#    Pool      ->(?, 8, 8, 64)
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

# L4
W4 = tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=0.01))
#    Conv      ->(?, 8, 8, 64)
#    Pool      ->(?, 4, 4, 64)
L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
L4 = tf.nn.relu(L4)
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L4_flat = tf.reshape(L4, [-1, 4 * 4 * 64])


# Final FC 4x4x64 inputs -> 2 outputs
W5 = tf.get_variable("W5", shape=[4 * 4 * 64, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([2]))
logits = tf.matmul(L4_flat, W5) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y_one_hot))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for i in range(100):
        feed_dict = {X: x_data, Y: y_data}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / 100

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: x_data_test, Y: y_data_test}))

# # Get one and predict
# r = random.randint(0, 10)
# print("Label: ", y_data_test[r])
# print("Prediction: ", sess.run(
#     tf.argmax(logits, 1), feed_dict={X: x_data_test[r]}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()

W0828 22:47:56.271350 11868 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0828 22:47:56.294290 11868 deprecation.py:323] From <ipython-input-1-a2664e809588>:74: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Learning started. It takes sometime.
Epoch: 0001 cost = 0.328650776
Epoch: 0002 cost = 0.000187518
Epoch: 0003 cost = 0.000043391
Epoch: 0004 cost = 0.000023711
Epoch: 0005 cost = 0.000014837
Epoch: 0006 cost = 0.000010077
Epoch: 0007 cost = 0.000007263
Epoch: 0008 cost = 0.000005474
Epoch: 0009 cost = 0.000004260
Epoch: 0010 cost = 0.000003408
Epoch: 0011 cost = 0.000002783
Epoch: 0012 cost = 0.000002305
Epoch: 0013 cost = 0.000001935
Epoch: 0014 cost = 0.000001644
Epoch: 0015 cost = 0.000001410
Epoch: 0016 cost = 0.000001220
Epoch: 0017 cost = 0.000001063
Epoch: 0018 cost = 0.000000930
Epoch: 0019 cost = 0.000000817
Epoch: 0020 cost = 0.000000722
Epoch: 0021 cost = 0.000000640
Epoch: 0022 cost = 0.000000571
Epoch: 0023 cost = 0.000000509
Epoch: 0024 cost = 0.000000456
Epoch: 0025 cost = 0.000000409
Epoch: 0026 cost = 0.000000367
Epoch: 0027 cost = 0.000000332
Epoch: 0028 cost = 0.000000298
Epoch: 0029 cost = 0.000000271
Epoch: 0030 cost = 0.000000248
Epoch: 0031 cost = 0.000000227
Ep

In [2]:
print("Label: ", y_data_test)
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: x_data_test}))

Label:  [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
Prediction:  [1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1]
